In [107]:
import requests
from lxml import etree
from bs4 import BeautifulSoup
import json
import time

In [34]:
login_url = 'https://accounts.douban.com/passport/login'
post_url = 'https://accounts.douban.com/j/mobile/login/basic'
toplist_url = 'https://movie.douban.com/top250?start=0&filter='

headers = {
    'Host': 'accounts.douban.com',
    'Referer': 'https://accounts.douban.com/passport/login',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
}

toplist_headers = {
    'Host': 'movie.douban.com',
    'Referer': 'https://movie.douban.com/top250',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
}

In [19]:
session = requests.Session()
response = session.get(login_url, headers=headers)

In [20]:
post_data = {
    'ck': '',
    'name': '18811319022',
    'password': 'JJkCE3CgsJJ.G!X',
    'remember': 'false',
    'ticket': ''}
login_response = session.post(post_url, data=post_data, headers=headers)

In [103]:
login_response.status_code

200

In [104]:
page_urls = []
for index in range(0, 250, 25):
    toplist_response = session.get('https://movie.douban.com/top250?start={}&filter='.format(index), headers=toplist_headers)
    if toplist_response.status_code == 200:
        soup = BeautifulSoup(toplist_response.text, 'html.parser')
        detailed_page_url = [_['href'] for _ in soup.select('.grid_view li .pic a')]
        page_urls.extend(detailed_page_url)

In [118]:
i = 0
info_dict_list = []

In [131]:
while i < len(page_urls):
    print('Current {:3d}/{:3d}'.format(i+1, len(page_urls)), end=', ')
    detailed_page_response = session.get(page_urls[i], headers=toplist_headers)
    if detailed_page_response.status_code == 200:
        print('200 OK')
        movie_soup = BeautifulSoup(detailed_page_response.text, 'html.parser')
        info_dict = json.loads(movie_soup.select('script[type="application/ld+json"]')[0].text, strict=False)
        info_list = [i.string for i in movie_soup.select('div#info')[0].children]
        for _, s in enumerate(info_list):
            if s == '制片国家/地区:':
                break
        info_dict['area'] = info_list[_+1]
        desc = list(movie_soup.find('div', attrs={'class': 'indent', 'id': 'link-report'}).children)[3].text.replace('\n', '').replace('\u3000', '').strip()
        info_dict['description'] = desc
        info_dict_list.append(info_dict)
    i = i + 1
#     time.sleep(3)

Current  20/250, 200 OK
Current  21/250, 200 OK
Current  22/250, 200 OK
Current  23/250, 200 OK
Current  24/250, 200 OK
Current  25/250, 200 OK
Current  26/250, 200 OK
Current  27/250, 200 OK
Current  28/250, 200 OK
Current  29/250, 200 OK
Current  30/250, 200 OK
Current  31/250, 200 OK
Current  32/250, 200 OK
Current  33/250, 200 OK
Current  34/250, 200 OK
Current  35/250, 200 OK
Current  36/250, 200 OK
Current  37/250, 200 OK
Current  38/250, 200 OK
Current  39/250, 200 OK
Current  40/250, 200 OK
Current  41/250, 200 OK
Current  42/250, 200 OK
Current  43/250, 200 OK
Current  44/250, 200 OK
Current  45/250, 200 OK
Current  46/250, 200 OK
Current  47/250, 200 OK
Current  48/250, 200 OK
Current  49/250, 200 OK
Current  50/250, 200 OK
Current  51/250, 200 OK
Current  52/250, 200 OK
Current  53/250, 200 OK
Current  54/250, 200 OK
Current  55/250, 200 OK
Current  56/250, 200 OK
Current  57/250, 200 OK
Current  58/250, 200 OK
Current  59/250, 200 OK
Current  60/250, 200 OK
Current  61/250,

In [133]:
len(info_dict_list)

250

In [136]:
lines = [json.dumps(info_dict, ensure_ascii=False) for info_dict in info_dict_list]

In [142]:
for line in lines:
    try:
        line.encode()
    except:
        print('Encode failed')
        break

In [149]:
line.encode().decode()

'{"@context": "http://schema.org", "name": "变脸 Face/Off", "url": "/subject/1292659/", "image": "https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2173855883.jpg", "director": [{"@type": "Person", "url": "/celebrity/1027853/", "name": "吴宇森 John Woo"}], "author": [{"@type": "Person", "url": "/celebrity/1300707/", "name": "麦克·韦柏 Mike Werb"}, {"@type": "Person", "url": "/celebrity/1281048/", "name": "迈克尔·科拉里 Michael Colleary"}], "actor": [{"@type": "Person", "url": "/celebrity/1054426/", "name": "约翰·特拉沃尔塔 John Travolta"}, {"@type": "Person", "url": "/celebrity/1054455/", "name": "尼古拉斯·凯奇 Nicolas Cage"}, {"@type": "Person", "url": "/celebrity/1025215/", "name": "琼·艾伦 Joan Allen"}, {"@type": "Person", "url": "/celebrity/1009310/", "name": "亚历桑德罗·尼沃拉 Alessandro Nivola"}, {"@type": "Person", "url": "/celebrity/1017913/", "name": "吉娜·格申 Gina Gershon"}, {"@type": "Person", "url": "/celebrity/1025191/", "name": "多米尼克·斯万 Dominique Swain"}, {"@type": "Person", "url": "/celebrity/1017986/"

In [151]:
with open('movie_data.txt', 'wt',encoding='utf-8', errors='replace') as f:
    f.writelines([json.dumps(info_dict, ensure_ascii=False).encode().decode()+'\n' for info_dict in info_dict_list])

In [ ]:
detailed_page = session.get(detailed_page_url[0], headers=toplist_headers)

In [ ]:
movie_soup = BeautifulSoup(detailed_page.text, 'html.parser')
info_dict = json.loads(movie_soup.select('script[type="application/ld+json"]')[0].text)
info_dict

In [ ]:
info_list = [i.string for i in movie_soup.select('div#info')[0].children]
for i, s in enumerate(info_list):
    if s == '制片国家/地区:':
        break
info_dict['area'] = info_list[i+1]

In [ ]:
if response.status_code == 200:
    self.dynamics(response.text)

In [ ]:
class Login(object):
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36',
            'Host': 'accounts.douban.com'
        }
        self.login_url = 'https://accounts.douban.com/j/mobile/login/basic'
        self.post_url = 'https://github.com/session'
        self.logined_url = 'https://github.com/settings/profile'
        self.session = requests.Session()
    
    def token(self):
        response = self.session.get(self.login_url, headers=self.headers)
        selector = etree.HTML(response.text)
        token = selector.xpath('//div//input[2]/@value')
        return token
    
    def login(self, email, password):
        post_data = {
            'commit': 'Sign in',
            'utf8': '✓',
            'authenticity_token': self.token()[0],
            'login': email,
            'password': password
        }
        response = self.session.post(self.post_url, data=post_data, headers=self.headers)
        if response.status_code == 200:
            self.dynamics(response.text)
        
        response = self.session.get(self.logined_url, headers=self.headers)
        if response.status_code == 200:
            self.profile(response.text)
    
    def dynamics(self, html):
        selector = etree.HTML(html)
        dynamics = selector.xpath('//div[contains(@class, "news")]//div[contains(@class, "alert")]')
        for item in dynamics:
            dynamic = ' '.join(item.xpath('.//div[@class="title"]//text()')).strip()
            print(dynamic)
    
    def profile(self, html):
        selector = etree.HTML(html)
        name = selector.xpath('//input[@id="user_profile_name"]/@value')[0]
        email = selector.xpath('//select[@id="user_profile_email"]/option[@value!=""]/text()')
        print(name, email)


if __name__ == "__main__":
    login = Login()
    login.login(email='cqc@cuiqingcai.com', password='password')